# Invest Partner - Seu parceiro de Investimentos

### Imports e APIKey

In [1]:
import openai
import time
from dotenv import load_dotenv, find_dotenv
import json
import yfinance as yf

In [2]:
_ = load_dotenv(find_dotenv())

client = openai.Client()

# Funções e Tools

In [3]:
def retorna_cotacao_acao_historica(
        ticker,
        periodo='1mo'
):
    
    ticker = yf.Ticker(ticker)
    hist = ticker.history(period=periodo)['Close']
    hist.index = hist.index.strftime('%Y-%m-%d')  # Formata o índice do DataFrame hist para strings no formato 'AAAA-MM-DD'.
    hist = round(hist, 2)   
    
    # Verifica se o DataFrame tem mais de 30 registros. Se tiver,  calcula um slice_size (tamanho fatia) dividindo por 30.
    # E reduz tamanho pegando um registro a cada slice_size, começando do final e revertendo para manter na sequência original.
    
    if len(hist) > 30:
        slice_size = int(len(hist) / 30)
        hist = hist.iloc[::-slice_size][::-1]
    
        
    return hist.to_json() # Passar como dicionario, e não como DF






def retorna_info(ticker):
    
    ticker = yf.Ticker(ticker)
    info = str(ticker.info)
        
    return info




def retorna_metadados(ticker, periodo):
    
    ticker = yf.Ticker(ticker)
    hist = ticker.history(period=periodo)
    metadados = str(ticker.history_metadata)
        
    return metadados





def retorna_noticias(ticker):
    ticker = yf.Ticker(ticker)
    noticias = str(ticker.news)
    
    return noticias




def retorna_desdobramentos(ticker):
    ticker = yf.Ticker(ticker)
    desdobramentos = ticker.splits
    
    return desdobramentos.to_json()



In [4]:
# Mapeando função
funcoes_disponiveis = {'retorna_cotacao_acao_historica': retorna_cotacao_acao_historica,
                      'retorna_info': retorna_info,
                      'retorna_metadados': retorna_metadados,
                       'retorna_noticias': retorna_noticias,
                       'retorna_desdobramentos': retorna_desdobramentos
                      }

### Tools

In [5]:
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'retorna_cotacao_acao_historica',
            'description': 'Retorna a cotação diária histórica para uma ação da bovespa',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3.SA" para ambev, "PETR4.SA" para petrobras, etc'
                    },
                    'periodo': {
                        'type': 'string',
                        'description': 'O período que será retornado de dados históriocos \
                                        sendo "1mo" equivalente a um mês de dados, "1d" a \
                                        1 dia e "1y" a 1 ano',
                        'enum': ["1d","5d","1mo","6mo","1y","5y","10y","ytd","max"]  # API so aceita esses dias
                    }
                }
            }
        }
    },
    
    
    
    
    
     {
        'type': 'function',
        'function': {
            'name': 'retorna_info',
            'description': 'Retorna informações gerais sobre uma ação, incluindo uma variedade de dados, \
            como o nome da empresa, setor da indústria, descrição da empresa, país de origem, e mais. É útil para obter \
            uma visão geral rápida e detalhes básicos sobre a empresa associada ao ticker fornecido.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3.SA" para ambev, "PETR4.SA" para petrobras, etc'
                    }
                }
            }
        }
    },
    
    
    {
        'type': 'function',
        'function': {
            'name': 'retorna_metadados',
            'description': 'Fornece informações, de acordo com a data fornecida, sobre os dados históricos disponíveis para \
            uma ação, incluindo o intervalo de datas disponíveis, os tipos de preços incluídos  (como abertura, fechamento, \
            máximos, mínimos e volume), divisões de ações, ajustes de dividendos e outros eventos corporativos relevantes. \
            Essas informações são úteis para entender a qualidade e o escopo dos dados históricos disponíveis.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3.SA" para ambev, "PETR4.SA" para petrobras, etc'
                    },
                    'periodo': {
                        'type': 'string',
                        'description': 'O período que será retornado de dados históriocos \
                                        sendo "1mo" equivalente a um mês de dados, "1d" a \
                                        1 dia e "1y" a 1 ano',
                        'enum': ["1d","5d","1mo","6mo","1y","5y","10y","ytd","max"]  # API so aceita esses dias
                    }
                }
            }
        }
    },
    
    

    
    
    
    
    {
        'type': 'function',
        'function': {
            'name': 'retorna_noticias',
            'description': 'retorna uma lista de notícias recentes relacionadas à empresa. Ele fornece manchetes, datas e links para artigos sobre a empresa cujas ações são negociadas na bolsa',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3.SA" para ambev, "PETR4.SA" para petrobras, etc'
                    }
                }
            }
        }
    },
    
    
    {
        'type': 'function',
        'function': {
            'name': 'retorna_desdobramentos',
            'description': 'retorna uma série temporal contendo os históricos de desdobramentos (splits) de ações de uma empresa. Ele fornece as datas e as razões dos splits ocorridos ao longo do tempo para uma determinada ação listada na bolsa.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3.SA" para ambev, "PETR4.SA" para petrobras, etc'
                    }
                }
            }
        }
    },
    
    
    {'type': 'code_interpreter'}
    
    
]


# Fine Tuning


import json

with open('chatbot_respostas.json', encoding="utf8") as f:
    json_respostas = json.load(f)


In [6]:
assistant_id = "asst_D72a8kRFXV99YSw4x8zKxS1Z"

In [7]:
thread = client.beta.threads.create()

In [8]:
def gera_resposta(mensagens):
    

    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role='user',
        content = mensagens
    )

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
        instructions='Seja breve e conciso na resposta'  # Se tivesse privilegios Premuim, concederia.
    )


    while run.status in ['queued', 'in_progress', 'cancelling']:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )


    if run.status == 'completed':
        messages = client.beta.threads.messages.list(
        thread_id=thread.id
      )
        print(messages)


    tool_outputs = []

    tool_calls = run.required_action.submit_tool_outputs.tool_calls

    if tool_calls:
        for tool in tool_calls:
            func_name = tool.function.name
            function_to_call = funcoes_disponiveis[func_name]
            func_args = json.loads(tool.function.arguments)
            func_return = function_to_call(**func_args)  # Passa todos os parametros do 'func_args', para a function_to_call
            tool_outputs.append({
                'tool_call_id': tool.id,
                'output': func_return
            })


    # Submit all tool outputs at once after collecting them in a list
    if tool_outputs:
        try:
            run = client.beta.threads.runs.submit_tool_outputs_and_poll(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
        except Exception as e:
            print("Failed to submit tool outputs:", e)

    if run.status == 'completed':
        messages = client.beta.threads.messages.list(
        thread_id=thread.id
      )
        print(messages.data[0].content[0].text.value) 
    else:
        print(run.status)


### Chat

In [9]:
if __name__ == '__main__':

    print('Olá, Seja bem-vindo ao Invest Partner, o seu parceiro de Investimentos :)')
    while True:
        input_usuario = input('User: ')
        mensagens = gera_resposta(input_usuario)

Olá, Seja bem-vindo ao Invest Partner, o seu parceiro de Investimentos :)
User: cotação atual petr4
A cotação atual da ação PETR4 é de R$ 35,82.



KeyboardInterrupt

